In [ ]:
!pip install streamlit streamlit-jupyter

In [6]:
%%writefile app.py
import os
import time
import requests
import streamlit as st
from dotenv import load_dotenv
load_dotenv()


# Configuration: Set up the base URL and API key header for the validation service.
# These values are taken from environment variables 
BASE_URL = os.getenv("VALIDATION_API_BASE_URL")
API_KEY_HEADER = os.getenv("VALIDATION_API_KEY_HEADER")

# Streamlit page CSS styling
st.markdown(
    """
    <style>
    .stApp {
        background-color: #231d4f;
    }
    .sidebar .sidebar-content {
        background-color: #231d4f;
        color: white;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Function: create_validation
def create_validation(api_key):
    """
    Create a new document validation request.
    Sends a POST request to the validation API.
    """
    headers = {
        API_KEY_HEADER: api_key,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "type": "document-validation",
        "country": "MX",
        "document_type": "national-id",
        "user_authorized": "true"
    }
    
    try:
        response = requests.post(BASE_URL, headers=headers, data=data, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        st.error(f"API error: {e.response.text}")
        return None
    except Exception as e:
        st.error(f"General error: {str(e)}")
        return None

# Function: upload_image
def upload_image(url, image_file):
    """
    Uploads an image to the specified URL using HTTP PUT.
    """
    try:
        if not url:
            st.error("Invalid URL for image upload")
            return False
            
        image_bytes = image_file.getvalue()
        response = requests.put(url, data=image_bytes)
        return response.status_code == 200
    except Exception as e:
        st.error(f"Error uploading image: {str(e)}")
        return False

# Function: get_validation_results
def get_validation_results(validation_id, api_key):
    """
    Polls the validation API to get the results of the document validation.
    """
    headers = {API_KEY_HEADER: api_key}
    url = f"{BASE_URL}/{validation_id}"
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        st.error(f"Error fetching results: {str(e)}")
        return None

    
# Streamlit User Interface
st.title("Mexican INE Validation")

# Input fields
api_key = st.text_input("Validation API Key", type="password", value="")
front_img = st.file_uploader("Upload Front of INE", type=["jpg", "jpeg", "png"])
back_img = st.file_uploader("Upload Back of INE", type=["jpg", "jpeg", "png"])

# Display INE images 
if front_img and back_img and api_key:
    col1, col2 = st.columns(2)
    with col1:
        st.image(front_img, caption="Front", width=250)
    with col2:
        st.image(back_img, caption="Back", width=250)
        
    #Trigger create_validation function
    if st.button("Validate Document"):
        with st.spinner("Creating validation..."):
            validation_data = create_validation(api_key)
            
        if validation_data:
            validation_id = validation_data.get("validation_id")
            front_url = validation_data.get("instructions", {}).get("front_url")
            back_url = validation_data.get("instructions", {}).get("reverse_url")

            if not all([front_url, back_url]):
                st.error("Upload URLs not generated. Please verify that the parameters are correct.")
                st.stop()

            # Upload front and back images
            if upload_image(front_url, front_img) and upload_image(back_url, back_img):
                # Placeholders for displaying status and results in real time
                status_placeholder = st.empty()
                status_value_placeholder = st.empty()
                results_placeholder = st.empty()
                
                status_placeholder.success("Images uploaded. Processing...")

                MAX_ATTEMPTS = 30
                WAIT_TIME = 30  
                final_status = None
                declined_reason = None
                final_results = None

                # Check API response for validation results
                for attempt in range(MAX_ATTEMPTS):
                    results = get_validation_results(validation_id, api_key)
                    
                    if results:
                        new_status = results.get("validation_status", "pending")
                        failure_status = results.get("failure_status")
                        current_reason = results.get("declined_reason")
                        
                        # Update the current status display
                        status_value_placeholder.markdown(
                            f"<h3 style='text-align: center; color: white;'>🔄 Current status: <strong>{new_status.upper()}</strong></h3>", 
                            unsafe_allow_html=True
                        )
                        
                        # Show progress information
                        results_placeholder.subheader(f"Progress (Attempt {attempt+1}/{MAX_ATTEMPTS})")
                        results_placeholder.json(results)
                        
                        final_results = results
                        
                        # Check for final statuses
                        if new_status in ["success", "failure"] or failure_status:
                            final_status = new_status if new_status in ["success", "failure"] else failure_status
                            declined_reason = current_reason
                            break
                    
                    time.sleep(WAIT_TIME)

                # Clear the status placeholders
                status_placeholder.empty()
                status_value_placeholder.empty()
                results_placeholder.empty()

                # Build the final result message based on the outcome
                final_title = "Final Result"
                status_style = ""
                details_message = ""
                
                if final_status == "success":
                    final_title = "✅ Validation Successful"
                    status_style = "color: #00ff00;"
                elif final_status == "failure":
                    final_title = "❌ Validation Failed"
                    status_style = "color: #ff0000;"
                    if declined_reason:
                        details_message = f"Reason: {declined_reason.replace('_', ' ').title()}"
                elif final_status == "expired":
                    final_title = "⌛ Time Expired"
                    status_style = "color: #ff9900;"
                else:
                    final_title = "⚠️ Unknown Status"
                    status_style = "color: #ababab;"

                # Display the final result
                st.markdown(
                    f"<h2 style='text-align: center; {status_style}'>{final_title}</h2>", 
                    unsafe_allow_html=True
                )
                
                if details_message:
                    st.markdown(
                        f"<h4 style='text-align: center; color: #ff6666;'>{details_message}</h4>",
                        unsafe_allow_html=True
                    )
                
                if final_results:
                    st.subheader("Complete validation details:")
                    st.json(final_results)
                else:
                    st.error("Failed to retrieve final results")
            else:
                st.error("Error uploading images")
        else:
            st.error("Error creating validation")


Writing app.py


In [ ]:
if __name__ == "__main__":
    import threading
    import webbrowser
    import time

    def open_browser():
        time.sleep(3)  # Open new browser window with port local host
        webbrowser.open("http://localhost:8501")

    # Call open browser function and run streamlit app in port
    threading.Thread(target=open_browser).start()
    !streamlit run app.py --server.port 8501 --server.headless true
